# Optimizing the Location of New Restaurants in Toronto, CA

### I. Background Information and Business Problem

Several criteria are involved in the process of choosing a location to start a restaurant. In major cities like Toronto, Canada, a restaurant owner has to be particularly aware of the following factors:

1. Their target demographic
2. Whether or not their restaurant is likely to be frequented by the local population
3. Effective marketing strategies to boost public awareness of their venue
4. The existing competitors to the cuisine offered

Given the relative difficulty of this task, the purpose of this study is to aid individuals who are planning to open a new restaurant in Toronto chose the right location by both providing and analyzing data relating to:

1. The per capita income of a given neighborhood
2. The population of a given neighborhood 
2. The competitors already present within a given neighborhood

Collectively, the findings in this report will not only enable restaurant owners to establish a restaurant in the right location, but it will also ensure the restaurant's profitability in the long term.

### II. Analytical Approach and Data Evaluation

In order to make appropriate recommendations to restaurant owners about the ideal location to start a new restaurant, data on the restaurant's competitors and target clientele will be needed. Of the factors relating to consumers, the biggest questions restaurant owners will need to answer are (1) whether or not the consumer is likely to eat at the restaurant based on their disposable income, and (2) whether there is a sufficiently large enough population in the neighborhood to attract a diverse customer base. The prediction is that the more wealthy a family is within a given neighborhood, the more likely they are to eat out at the restaurant. Moreover, it is expected that the more people there are in the neighborhood, the probability that the restaurant will attract more customers will be greater. 

Separately, looking at factors that do not pertain to consumers themselves, a restaurant owner will also have to consider the potential competitors to their business model based on whether there are similar restaurants offering the same cuisine as the new restaurant. If there are, then it is expected that the restaurant will not be as profitable in the given location.

In order to evaluate these hypotheses, data on per capita income for a given neighborhood as well as the population of each neighborhood will be needed; this will come from Canadian census data for the city of Toronto. Additionally, a list of restaurants and their cuisines will be needed for each neighborhood to evaluate the threat competitors pose the the new restaurant's business model; this data will come from the Foursquare location database that is accessible online.